<a href="https://colab.research.google.com/github/edquestofficial/Gen-AI-Cohort/blob/main/1_day/1_chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import re
import pandas as pd
from google.colab import drive

In [ ]:
# def mount_drive():
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def fix_size_text_chunker(text, chunk_size):
    """
    Splits text into chunks based on the specified chunk size.

    Args:
        text (str): The input text to be chunked.
        chunk_size (int): The maximum size of each chunk.

    Returns:
        list: A list of text chunks.
    """
    chunks = []  # List to store the text chunks
    words = text.split()  # Split the text into words

    current_chunk = []  # Store words for the current chunk
    current_chunk_length = 0  # Total length of words in the current chunk

    # Iterate through each word in the text
    for word in words:
        # Check if adding the current word to the chunk exceeds the chunk size
        if current_chunk_length + len(word) + 1 <= chunk_size:
            current_chunk.append(word)
            current_chunk_length += len(word) + 1
        else:
            # Add the current chunk to the list of chunks
            chunks.append(' '.join(current_chunk))
            # Start a new chunk with the current word
            current_chunk = [word]
            current_chunk_length = len(word)

    # Append the remaining chunk if it's not empty
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [ ]:
def fix_size_chunking(page,pdf_meta, chunk_size):
  # Initialize an empty list to store the text chunks
  text_chunks = []
  all_dfs = []
  text = page.extract_text()

  # Remove blank lines and tables
  text = re.sub(r'^\s*$\n', '', text, flags=re.MULTILINE)  # Remove blank lines
  text = re.sub(r'\|?\s*\|\s*\|?', '', text)  # Remove table lines
  words = text.split()  # Split the text into words
  print(f"length of text: {len(words)}")

  text_chunks = fix_size_text_chunker(text, chunk_size)

  # Create a DataFrame to store the chunks and page title
  data = {'Title': [], 'Chunk Text': []}
  for idx, chunk in enumerate(text_chunks):
    data['Title'].append(pdf_meta.author)
    data['Chunk Text'].append(chunk)
  return data


In [ ]:
def extract_sentences_from_text(text):
    """
    Extracts sentences from the input text.

    Args:
        text (str): The input text containing multiple sentences.

    Returns:
        list: A list of sentences extracted from the text.
    """
    sentences = text.split('\n')  # Split the text into lines to extract sentences
    return sentences

In [ ]:
def sentence_chunking(pdf_page, pdf_meta):
    """
    Extracts and chunks sentences from a PDF page.

    Args:
        pdf_page (str): The text extracted from the PDF page.
        pdf_meta (str): Metadata of the PDF.

    Returns:
        dict: A dictionary containing page title and chunked text.
    """
    # Extract text from the PDF page
    text = pdf_page.extract_text()

    # Remove blank lines from the text
    text = re.sub(r'^\s*$\n', '', text, flags=re.MULTILINE)

    # Remove table lines from the text
    text = re.sub(r'\|?\s*\|\s*\|?', '', text)

    # Split the text into sentences
    sentence_chunks = extract_sentences_from_text(text)

    # Create a dictionary to store the chunks and page title
    data = {'Title': [], 'Chunk Text': []}

    # Add page title and chunked text to the dictionary
    for idx, chunk in enumerate(sentence_chunks):
        data['Title'].append(pdf_meta.author)
        data['Chunk Text'].append(chunk)

    return data

In [ ]:
def extract_text_from_page(pdf_page):
    """
    Extracts text from a PDF page.

    Args:
        pdf_page: The PDF page to extract text from.

    Returns:
        str: The extracted text from the PDF page.
    """
    return pdf_page.extract_text()

In [ ]:
def page_chunking(pdf_page, pdf_meta):
    """
    Extracts and chunks text from a PDF page.

    Args:
        pdf_page: The PDF page to extract text from.
        pdf_meta: Metadata of the PDF.

    Returns:
        dict: A dictionary containing page title and chunked text.
    """
    # Extract text from the PDF page
    text = extract_text_from_page(pdf_page)

    # Remove blank lines and table lines from the text
    cleaned_text = re.sub(r'^\s*$\n', '', text, flags=re.MULTILINE)  # Remove blank lines
    cleaned_text = re.sub(r'\|?\s*\|\s*\|?', '', cleaned_text)  # Remove table lines

    # Create a dictionary to store the chunks and page title
    data = {'Title': [], 'Chunk Text': []}

    # Add page title and chunked text to the dictionary
    data['Title'].append(pdf_meta.author)
    data['Chunk Text'].append(cleaned_text)

    return data

In [ ]:
# Open the PDF file
pdf_file = open('/content/drive/MyDrive/Gen Ai/upgrad/data/bike2024-2025.pdf', 'rb')
fix_all_dfs = []
sentence_all_dfs = []
page_all_dfs = []
chunk_size = 300
# Create a PDF reader object
pdf_reader = PyPDF2.PdfReader(pdf_file)
# Number of pages in the PDF
pdf_pages_count = len(pdf_reader.pages)
pdf_meta = pdf_reader.metadata
print(f"Number of pages in pdf {pdf_pages_count}")
for page_num in range(pdf_pages_count):
  # Extract the text from the current page
  text = pdf_reader.pages[page_num].extract_text()
  page = pdf_reader.pages[page_num]
  fix_chunk_data = fix_size_text_chunker(text, chunk_size)
  sentence_chunkdata = sentence_chunking(page, pdf_meta)
  page_chunk_data = page_chunking(page, pdf_meta)
  # all_dfs.append(pd.DataFrame(data))
  print(f"pange no: {page_num}  && length of df : {len(fix_all_dfs)}")
  fix_all_dfs.append(pd.DataFrame(fix_chunk_data))
  sentence_all_dfs.append(pd.DataFrame(sentence_chunkdata))
  page_all_dfs.append(pd.DataFrame(page_chunk_data))
fix_chunk_df = pd.concat(fix_all_dfs, ignore_index=True)
sentence_chunk_df = pd.concat(sentence_all_dfs, ignore_index=True)
page_chunk_df = pd.concat(page_all_dfs, ignore_index=True)
pdf_file.close()
print(f"fix_chunk_df.shape : {fix_chunk_df.shape}")
print(f"sentence_chunk_df : {sentence_chunk_df.shape}")
print(f"page_chunk_df.shape : {page_chunk_df.shape}")
print("======================================================")
print(fix_chunk_df.head())
print("======================================================")
print(sentence_chunk_df.head())
print("======================================================")
print(page_chunk_df.head())

Number of pages in pdf 2
pange no: 0  && length of df : 0
pange no: 1  && length of df : 1
fix_chunk_df.shape : (44, 1)
sentence_chunk_df : (133, 2)
page_chunk_df.shape : (2, 2)
                                                   0
0  Vehicle Details Policy Details Make ROYAL ENFI...
1  ME3U3S5C1HC817838 Invoice No. 204561186201000 ...
2  GURGAON TOWER B15 FLAT 804 AVL36GURGAON SECTOR...
3  0 47250(`) Own Damage Policy Period Liability ...
4  Third Party Liability 1366 Total Basic Premium...
       Title                                         Chunk Text
0  HDFC ERGO                     Vehicle Details Policy Details
1  HDFC ERGO  Make ROYAL ENFIELD Policy No. 2301 2045 6118 6...
2  HDFC ERGO      Model - Variant CLASSIC-Classic 350 Period of
3  HDFC ERGO               InsuranceFrom 15 Mar, 2024 00:01 hrs
4  HDFC ERGO  Registration No HR-26-DD-2318 To 14 Mar, 2025 ...
       Title                                         Chunk Text
0  HDFC ERGO  Vehicle Details Policy Details\nMake ROYAL